In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [3]:
# # 大庆市
# roi = ee.Geometry.Rectangle([123.9355, 45.8362,125.3055, 46.8762])
# # 天津市
# roi = ee.Geometry.Rectangle([115.9244, 38.6272,117.4944,39.6372])
# # 武汉市
# roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
# # 珠三角
# roi = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
# 青藏高原
roi = ee.Geometry.Rectangle([89.8826, 34.6579,91.3626, 35.8279])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

In [4]:
elevation = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(roi).reproject('EPSG:3857',None,30)
dataset_id = 'hillshade_tibet'
assetID = 'users/311605001111/' + dataset_id
task = ee.batch.Export.image.toAsset(**{
    'image': elevation,
    'description': dataset_id,
    'assetId': assetID,
    'scale': 30,
    'region': roi,
    'maxPixels': 1e13,
})
task.start()

In [ ]:
image = ee.Image("users/311605001111/hillshade_daqing").gte(0).multiply(ee.Image.pixelArea()).divide(1e6)
areas = image.reduceRegion(**{
    'reducer': ee.Reducer.sum(),
    'geometry': roi,
    'scale': 30,
    'maxPixels': 1e14
})
print(areas.getInfo())
Map.addLayer(ee.Image("users/311605001111/hillshade_daqing"), {'min': 0, 'max': 200}, "image")

l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
       .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi) \
       .filterDate('2020-01-01', '2020-12-31')          
# Map.addLayer(l8.map(maskSR).mosaic().clip(roi), visParams,'image mosaic')
areas = l8.mosaic().clip(roi).select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6).reduceRegion(**{
    'reducer': ee.Reducer.sum(),
    'geometry': roi,
    'scale': 30,
    'maxPixels': 1e14
})
print(areas.getInfo())